Baseline - Tn Promedio de 12 meses

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress
import os

# Optional for larger text and better visuals
sns.set(style="whitegrid", context="talk")

In [3]:
# Load the sales data (tab-delimited)
sales = pd.read_csv("datasets/sell-in.txt", sep="\t", dtype={"periodo": str})

# Load the stocks data (tab-delimited)
stocks = pd.read_csv("datasets/tb_stocks.txt", sep="\t", dtype={"periodo": str})

# Load the product information data (tab-delimited)
product_info = pd.read_csv("datasets/tb_productos.txt", sep="\t")

#carga productos a predecir
product_predict = pd.read_csv("datasets/product_id_apredecir201912.txt", sep="\t",header=0)

In [4]:
# Asegurar que 'periodo' sea string
sales['periodo'] = sales['periodo'].astype(str)



In [5]:
# Paso 1: agrupar por producto y periodo, sumando tn por mes
tn_por_producto_y_periodo = (
    sales
    .groupby(['product_id', 'periodo'])['tn']
    .sum()
    .reset_index()
)


In [6]:
# Paso 2: filtrar solo los últimos 3 meses deseados
ultimos_12 = ['201901', '201902', '201903','201904', '201905', '201906','201907', '201908', '201909','201910', '201911', '201912']
tn_filtrado = tn_por_producto_y_periodo[tn_por_producto_y_periodo['periodo'].isin(ultimos_12)]

In [7]:
# Paso 3: calcular el promedio de esos tres meses por producto
promedio_ultimos12 = (
    tn_filtrado
    .groupby('product_id')['tn']
    .mean()
    .reset_index()
    .rename(columns={'tn': 'tn'})
)

In [8]:
# Unir con productos a predecir
submission = product_predict.merge(promedio_ultimos12, on='product_id', how='left')
submission['tn'] = submission['tn'].fillna(0)

In [9]:
submission.head()

,product_id,tn
0,20001,1454.732720
1,20002,1175.437142
2,20003,784.976407
3,20004,627.215328
4,20005,668.270104


In [10]:
submission.to_csv("data/pred_trivial_media12.csv", index=False, encoding="utf-8")